In [39]:
import pandas as pd
#importing week2 modified data
df = pd.read_csv('modified_churn_data.csv', index_col='customerID', )
#removing extra columns
df = df.drop(columns=['TotalRevenue','Log_MonthlyCharges'],axis=1)
#loading new data for predection
new_data = pd.read_csv('new_churn_data.csv')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,0,0,1,29.85,29.85,0
5575-GNVDE,34,1,1,2,56.95,1889.50,0
3668-QPYBK,2,1,0,2,53.85,108.15,1
7795-CFOCW,45,0,1,3,42.30,1840.75,0
9237-HQITU,2,1,0,1,70.70,151.65,1
...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,2,84.80,1990.50,0
2234-XADUH,72,1,1,4,103.20,7362.90,0
4801-JZAZL,11,0,0,1,29.60,346.45,0


In [40]:
from pycaret.classification import *

In [41]:
# Set up PyCaret with the reduced dataset (common columns)
automl = setup(df, target='Churn')

,Description,Value
0,Session id,578
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 7)"
4,Transformed data shape,"(7043, 7)"
5,Transformed train set shape,"(4930, 7)"
6,Transformed test set shape,"(2113, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [42]:
# Comparing models and choosing the best one based on AUC (Area Under Curve)
best_model = compare_models()

# Display the best model
print("Best Model Selected:", best_model)

# Saving the best model to disk
predict_model(best_model,df.iloc[-2:-1])


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:57:35
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7941,0.8324,0.5076,0.6424,0.5665,0.4340,0.4396,0.0290
ada,Ada Boost Classifier,0.7929,0.8319,0.5000,0.6418,0.5615,0.4288,0.4349,0.0640
ridge,Ridge Classifier,0.7899,0.8203,0.4373,0.6568,0.5245,0.3966,0.4104,0.0130
lda,Linear Discriminant Analysis,0.7899,0.8203,0.4939,0.6341,0.5547,0.4201,0.4260,0.0110
gbc,Gradient Boosting Classifier,0.7895,0.8316,0.4946,0.6323,0.5545,0.4195,0.4252,0.1410
lightgbm,Light Gradient Boosting Machine,0.7850,0.8209,0.4939,0.6196,0.5492,0.4104,0.4153,0.1140
rf,Random Forest Classifier,0.7710,0.7940,0.4701,0.5861,0.5209,0.3730,0.3774,0.1820
knn,K Neighbors Classifier,0.7613,0.7315,0.4266,0.5684,0.4869,0.3354,0.3416,0.0300
et,Extra Trees Classifier,0.7606,0.7729,0.4770,0.5569,0.5131,0.3559,0.3582,0.1200
qda,Quadratic Discriminant Analysis,0.7471,0.8210,0.7416,0.5170,0.6089,0.4309,0.4465,0.0140


Best Model Selected: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=578, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4,1,0,2,74.400002,306.600006,1,1,0.5596


In [43]:
save_model(best_model,'LR')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',...
                                                               fill_value=None,
                        

In [44]:
# Define the churn prediction function with inspection
def predict_churn(df, original_data):
    # Make predictions using the loaded model
    predictions = predict_model(best_model, data=new_data)
    # Print the columns to inspect the output
    print(predictions.columns)
    # Print the predictions dataframe to understand the column names
    print(predictions.head())
    # Return the churn probabilities and predicted labels with customerID
    return predictions

In [45]:
# Use the function to predict churn on the new data
predictions = predict_churn(df, new_data)

# Print the predictions
print(predictions)

Index(['customerID', 'tenure', 'PhoneService', 'Contract', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'charge_per_tenure',
       'prediction_label', 'prediction_score'],
      dtype='object')
   customerID  tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
0  9305-CKSKC      22             1         0              2       97.400002   
1  1452-KNGVK       8             0         1              1       77.300003   
2  6723-OKKJM      28             1         0              0       28.250000   
3  7832-POPKP      62             1         0              2      101.699997   
4  6348-TACGU      10             0         0              1       51.150002   

   TotalCharges  charge_per_tenure  prediction_label  prediction_score  
0    811.700012          36.895454                 1            0.5276  
1   1701.949951         212.743744                 1            0.6362  
2    250.899994           8.960714                 0            0.8680  
3   3106.560059    

In [46]:
# saving predection file to the disk
predictions.to_csv('predictions.csv', index=False)

In [47]:
# Reading the predection file
dfa = pd.read_csv('predictions.csv')
dfa.head()

,customerID,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,prediction_label,prediction_score
0,9305-CKSKC,22,1,0,2,97.40,811.70,36.895454,1,0.5276
1,1452-KNGVK,8,0,1,1,77.30,1701.95,212.743740,1,0.6362
2,6723-OKKJM,28,1,0,0,28.25,250.90,8.960714,0,0.8680
3,7832-POPKP,62,1,0,2,101.70,3106.56,50.105810,0,0.7841
4,6348-TACGU,10,0,0,1,51.15,3440.97,344.097000,1,0.7384


# Summary
By deploying PyCaret, I proceeded to utilise a myriad of machine learning models on the churn dataset that we refined. The feature selection criteria used were Based on statistical testing and mutual information values for the selected genotype-treatment interactions because it gave me good results during the model comparisons, while model comparison criteria were chosen based on a balanced measure of precision and recall given the imbalanced nature of the dataset. The best model was selected and this was written on disk so as to be used in the future. I then developed a Python script that loads the made and predicts the probability of churn on new data with the formula. When run on other data, the script was able to give the right probabilities predicting that customers who actually churned were the ones with the highest probabilities.The findings of the study indicated high validity of the best model with 0.83 AUC and 79% accuracy.